# Simulations for overlapping orders

*The main purpose of this notebook is to show how to use [Sherpa](https://cxc.cfa.harvard.edu/sherpa/) to simulate Arcus data, taking into account insufficient order-sorting but I hope that some of the plots and general discussion are also useful for users of other software packages. You can read this notebook as just a simple example of how Arcus data might look like and ignore all the code and associated discussion in between, read it as a how-to, or [download the notebook source](https://github.com/hamogu/arcus-analysis/blob/master/notebooks/SimMultiOrder.ipynb) and use it as a template for your own simulations.*

In [ ]:
from nbtemplate import display_header
display_header('SimMultiOrder.ipynb', status='Arcus Probe')

See [my memo on ARCUS order sorting](https://space.mit.edu/home/guenther/ARCUS/OrderSorting.html) for a high level of explanation why order sorting in Arcus is more messy than in e.g. Chandra/HETGS or XMM/RGS and how we can deal with that in principle. Here, I concentrate on what commands to type in to make the correct kind of simulation.

To aid users who read this notebook as a how-to for simulating Arcus data in [Sherpa](https://cxc.cfa.harvard.edu/sherpa/) or who download this notebook as a template, I add detailed descriptions of what the code does. Some of this is in text cells (like this one), some is placed as a comment directly in the code. **If you read this notebook on my website, I reccomend to click the blue button on the top to toggle on the code view.**

In [ ]:
import os
from os.path import join as pjoin
import numpy as np
import matplotlib.pyplot as plt

from sherpa.astro import ui
from sherpa.astro import datastack as ds

%matplotlib inline

In [ ]:
# Avoid double printing of certain sherpa messages in notebooks. 
# Might not be needed for all setups, but should also do no harm.
#import logging
#logging.getLogger('sherpa').propagate = 0

In [ ]:
# Set the path to the arf/rmf on my computer
arfrmfpath = '../../arfrmf_20230228/'

## 1 - Simulate a single order
The orders in Arcus are very close together in energy space (see [my memo on ARCUS order sorting](https://space.mit.edu/home/guenther/ARCUS/OrderSorting.html)), in fact so close that some photons will be sorted into the wrong orders. We provide ARF files for different order-sorting strategies that will lead to different effective areas and different different amounds of cross-talk.

First, I show how to simulate just one order (including the cross-talk that lands in there).

Let's start with a fairly simple model of a powerlaw plus a Gaussian.

<div style="background-color:orange;font-weight:bold">
If you want to simulate a single order for Arcus with Sherpa, put your model in the cell below.</div>

In [ ]:
mpow = ui.xspowerlaw('mpow')
mgau = ui.xsgaussian('mgau')
src = mpow + mgau
mgau.LineE = 0.5
mgau.norm = 1
mgau.sigma = 0.001
src

We will simulate order -5, in the `osiptouch` scenario. For that, we really need to simulate three spectra (the main order -5 and the true order -4 and -6 photons that erroneously get assigned to order -5 in the order-sorting process using the CCD energies). Thus, we need three ARFs (`far_chan_all_ccdord_-5_true_-5.arf`, `far_chan_all_ccdord_-5_true_-4.arf`, `far_chan_all_ccdord_-5_true_-6.arf`) and the associated RMFs (`far_chan_all_-5.rmf`,`far_chan_all_-4.rmf`,`far_chan_all_-6.rmf `). When we analyze such a dataset that contains contributions from multiple orders, we simply load all three ARFs and RMFs into the same dataset and Sherpa will do the right thing.

OK, now let's run the simulation. Of course, the directory names and path might differ in your case.

<div style="background-color:orange;font-weight:bold">
If you want to simulate a single order for Arcus with Sherpa, put the path to the ARFs and RMFs in here.</div>

In [ ]:
ui.set_source('pow_ord_-5_touch', src)
ui.fake_pha('pow_ord_-5_touch',
            # It's important to list the main order first, because that will be used
            # later to set the energy/wavelength scale for plotting.
           [arfrmfpath + 'osiptouch/far_chan_all_ccdord_-5_true_-4.arf',
            arfrmfpath + 'osiptouch/far_chan_all_ccdord_-5_true_-5.arf',
            arfrmfpath + 'osiptouch/far_chan_all_ccdord_-5_true_-6.arf'],
           [arfrmfpath + 'far_chan_all_-4.rmf',
            arfrmfpath + 'far_chan_all_-5.rmf',
            arfrmfpath + 'far_chan_all_-6.rmf'],
            exposure=1e3)

The plot above compared two ways to calculate the effective area. It shows a zoom in onto some wavelength region to show small details, in this wavelength region the diffraction orders -3 and -4 are more important. The effective area for both curves is shown with colored lines. The lower, more wiggly line of every color is calculated directly by summing up the photons for each simulation in a grid with a relatively narrow step size in wavelength and simulations run for all four channels. Because of that, the number of photons in each channel is set to be only 100,000; this relatively low number causes the wiggliness. Despite the smaller step size, certain features like the chip gaps in order -3 around 37 Ang are not entirely resolved. The wavelength grid misses the first two, while the latter two are shown.

Also, the directly calculated curve is significantly below the second curve of the same color. This second curve is written using the ARF/RMF formalism we apply here, with a semi-analytical description of the ARF. The ARF is build by interpolating a mirror/grating efficiency curve, which is based on ray-traces with a 1 Ang step size and contains the mirror efficiency, the geometric opening, the effect of the blaze angle, the size of the active area of the grating membranes etc. It does not include filters and QE (those are applied separately by `marxs.missions.arcus.arfrfm.mkarf`) and it does not include the position of the chips, which is calculated analytically and can thus be sharply resolved for any dense choice of energy grid. This second curve is higher, because it **does not include** the effect of the extraction width - all photons are included, no matter how far away they are in cross-dispersion direction. In the ARF/RMF formalism we use, this factor is included in the normalization of the RMF later; while the directly calculated ARF (the lower, wiggly line) already has some cross-dispersion filtering applied. (That means that this plot also shows how much we expect to loose from cross-dispersion.)

In [ ]:
# basic plotting with Sherpa functions
ui.set_analysis("wave")
ui.group_counts('pow_ord_-5_touch', 50)
ui.plot_fit('pow_ord_-5_touch', ylog=True)
for i in [1, 2, 3]:
    ui.plot_order('pow_ord_-5_touch', i, overplot=True)

# modify the plot with matplotlib itself
# That's easier for direct formatting.
ax = plt.gca()
ax.set_ylim(1e-2, None)
lines = ax.get_lines()
out = ax.set_title('Order -5, osiptouch')
out = ax.legend(handles=[lines[i] for i in [1,2, 4, 6, 8]],
                labels=['simulated data', 'model (total)', 
                        'CCD order -5', 'CCD order -4', 'CCD order -6'])

This plot shows the simulated data. Why are there three emission lines, when our input model had just one? The central line is the emission line seen in order -5 (green, hard to see on top of the blue data). However, in order -4 and -6 the same emission line gets diffracted to a different position on the detector and if we then resolve he events *as if the photons where order -5*, then we assign the wrong wavelength to the photons, leading to apparent emission lines in order locations. For the emission lines it's thus easy to see that the one in the middle if the emission line in order -5 and ones to the left and right are from order -6 and -4, respectively. Of course, the same happens for photons in the continuum, but there we do not have such a convenient way to say "this photon is from that true order". However, because the set up the (simulated) data correctly with all three contributing ARFs and RMFs, the total model (orange) "knows" about all contributions and describes all three apparent emission lines and the shape of the continuum.

Note that we chose to make the y-axis logarithmic to highlight the contributions from the confused orders, which dominate over the local continuum, but are much lower than the main emission line. On the left side (short wavelengths) the confusing orders contribute of order 1% of the flux, on the right hand side (long wavelength) that's up to 10%. Only of there are strong emission lines a situation arises where the confusing orders contribute more counts than the main order at a specific wavelength. However, even the low-level contribution from the continuum may influence fitted parameters such as the powerlaw slope of the level or the depth of an absorption line if not taken into account properly in the fitting.

We can select a narrower order-sorting region, which will cost us total effective area, but supress the confusing order more than the main order. 

I now repeat all the simulation steps above for a narrower order sorting region and redo the same plot. After that, I plot the two simulations on top of each other for comparison.

In [ ]:
# Same as above except that I changed the name of the simulated dataset
# and the path so the ARF files
ui.set_source('pow_ord_-5_osip60', src)
ui.fake_pha('pow_ord_-5_osip60',
         # It's important to list the main order first, because that will be used
         # later to set the energy/wavelength scale for plotting.
        [arfrmfpath + 'osip60/far_chan_all_ccdord_-5_true_-4.arf',
         arfrmfpath + 'osip60/far_chan_all_ccdord_-5_true_-5.arf',
         arfrmfpath + 'osip60/far_chan_all_ccdord_-5_true_-6.arf'],
        [arfrmfpath + 'far_chan_all_-4.rmf',
         arfrmfpath + 'far_chan_all_-5.rmf',
         arfrmfpath + 'far_chan_all_-6.rmf'],
        exposure=1e3)

In [ ]:
# basic plotting with Sherpa functions
ui.set_analysis("wave")
ui.group_counts('pow_ord_-5_osip60', 50)
ui.plot_fit('pow_ord_-5_touch', ylog=True)
for i in [1, 2, 3]:
    ui.plot_order('pow_ord_-5_osip60', i, overplot=True)

# modify the plot with matplotlib itself
# That's easier for direct formatting.
ax = plt.gca()
ax.set_ylim(1e-2, None)
lines = ax.get_lines()
out = ax.set_title('Order -5, osip 60 eV')
out = ax.legend(handles=[lines[i] for i in [1,2, 4, 6, 8]],
                labels=['simulated data', 'model (total)', 
                        'CCD order -5', 'CCD order -4', 'CCD order -6'])

In [ ]:
# basic plotting with Sherpa functions
ui.set_analysis("wave")
ui.group_counts('pow_ord_-5_osip60', 50)
ui.plot_fit('pow_ord_-5_touch', ylog=True)
#for i in [1, 2, 3]:
#    ui.plot_order('pow_ord_-5_touch', i, overplot=True)

ui.plot_fit('pow_ord_-5_osip60', overplot=True)
# modify the plot with matplotlib itself
# That's easier for direct formatting.
ax = plt.gca()
ax.set_ylim(1e-0, None)
lines = ax.get_lines()
out = ax.legend(handles=[lines[i] for i in [1, 2, 5, 6]],
                labels=['OSIP touch - data', 'OSIP touch - model', 
                        'OSIP 60 eV - data', 'OSIP 60 eV - model'])

The "OSIP touch" scenario uses a wide order-sorting region (see [my memo on ARCUS order sorting](https://space.mit.edu/home/guenther/ARCUS/OrderSorting.html) or the pdf files in the tar file distribution of responses), the "OSIP 60 eV" a rather narrow one. The overall flux in the continuum and the main emission line is about 40% lower with the narrow order-sorting region, but the contribution of the interloping orders is suppressed much more - a factor of ten on the left side of the order and still a factor 3 on the right side. Is this worth the reduced overall effective area? That depends on your science case.

Now, to finish of this section, let's print the ARFs and RMFs that we loaded to make one of the simulations, just to make sure we grapped the right files.

In [ ]:
ui.show_data('pow_ord_-5_touch')

We can now go and do some more analysis e.g. play with the normalization of the Gaussian and see how well a fit can recover the input parameters depending o nthe total count number.

## 2 - Simulate all orders
Arcus has numerous orders. We currently provide ARFs and RMFs from -11 to +2. Only for the orders on the "far" side we have the situation that the order-sorting regions overlap; for the 0th order and the "near" orders it is sufficient to load a single ARF and RMF per order. Still, a simulation of all orders takes 15 orders with > 50 ARF/RMF pairs (some files are used more than once because true order -5 photons contribute to the CCD extracted orders -4, -5, and -6). And that's only because we've already coadded all four channels in Arcus when we made the ARFs and RMFs. For now that's OK, but if after launch the calibration turns out to be subtly different, we may need > 200 ARF/RMF pairs to fit a single spectrum!

That's a lot of booking to do, and thus I've written a function below that will load all those files and loop over all orders to run the full simulation. It returns a "datastack" ([a Sherpa object that allows us to treat a number of datasets in one go](https://cxc.cfa.harvard.edu/sherpa/ahelp/datastack.html)).

In [ ]:
sidelist = [-1, 0, 1]

def fake_all_orders(src, arfrmfpath, farosipname, bkgpath=None, **kwargs):
    '''Simulate all  from a model.
    
    The function creates a simulated PHA data for all orders in an
    Arcus simulation. To make this work, it assumes that ARF and RMF
    file names follow a naming convention and are sorted in 
    directories the same way that we distribute them.
    
    This function generates a bunch of generically names Sherpa
    datasets, with names such as `near_-1`. If those datasets exisit,
    they are overwritten.

    Parameters
    ----------
    src : Sherpa model
        Shera model that is used to generate the simulated photons
    arfrmfpath : string
        base directory with ARF and RMF files that do not depend on
        order-sorting (e.g. 0th order)
    farosipname : string
        name of the directory that contains the ARF files that *do* depend
        on order-sorting. For example, this could be `farosipname='osiptouch'`.
    bkgpath : string or None
        If set, needs ot point to a directory that holds the background pha set.

    For all other parameters (exposure time, background, grouping etc.) see Sherpa's `fake_pha`
    docstring. All those parameters are simply passed though and will be applied 
    to each of the simulations.
    '''
  
    fake = ds.DataStack()
    ui.set_source('0', src)
    ui.fake_pha('0',
             pjoin(arfrmfpath, 'chan_all_+0.arf'),
             pjoin(arfrmfpath, 'chan_all_+0.rmf'), 
             **kwargs)
    fake._add_dataset('0')
    for o in ['+2', '+1', '-1', '-2']:
        ui.set_source(f'near_{o}', src)
        ui.fake_pha(f'near_{o}',
                 pjoin(arfrmfpath, f'near_chan_all_{o}.arf'),
                 pjoin(arfrmfpath, f'near_chan_all_{o}.rmf'),
                 **kwargs)
        fake._add_dataset(f'near_{o}')
    for o in np.arange(-11, -1):
        ui.set_source(f'far_{o}', src)
        arflist = [pjoin(arfrmfpath, farosipname, f'far_chan_all_ccdord_{o}_true_{o + t}.arf') for t in sidelist
                  if os.path.isfile(pjoin(arfrmfpath, farosipname, f'far_chan_all_ccdord_{o}_true_{o + t}.arf'))]
        rmflist = [pjoin(arfrmfpath, f'far_chan_all_{o + t}.rmf') for t in sidelist 
                  if os.path.isfile(pjoin(arfrmfpath, f'far_chan_all_{o + t}.rmf'))]
        ui.fake_pha(f'far_{o}',
                 arflist if len(arflist) > 1 else arflist[0],
                 rmflist if len(rmflist) > 1 else rmflist[0],
                 bkg=ui.unpack_pha(pjoin(arfrmfpath, f'arcus_nxr_bg_{o}.fits')) if bkgpath else None,
                 **kwargs)
        fake._add_dataset(f'far_{o}')
    return fake

For these simulations, we make a little more complicated model. The input spectral model is for the Herbig AeBe star HD 163296 and is taken from the XMM-Newton observation published in [Günther & Schmitt (2009)](https://ui.adsabs.harvard.edu/abs/2009A%26A...494.1041G). The model consits of three VAPEC components with non-solar abundances and an absorbing column.

<div style="background-color:orange;font-weight:bold">
If you use this notebook as a template for your own simulation, edit the cell below with your input model.</div>

In [ ]:
abs1 = ui.xsphabs('abs1')
apec1 = ui.xsvapec('vapec1')
apec2 = ui.xsvapec('vapec2')
apec3 = ui.xsvapec('vapec3')
src = abs1 * (apec1 + apec2 + apec3)

# tie all the abundances in all VAPEC components
for apec in  [vapec2, vapec3]:
    for i in np.arange(1, len(apec.pars) - 1):
        apec.pars[i].val = apec1.pars[i]

# Set abundances to the numbers published in Günther & Schmitt (2009)
ui.set_xsabund('grsa')
        
abs1.nH = 0.07
apec1.kT = 0.21
apec2.kT = 0.51
apec3.kT = 2.7
apec1.C = 3.7
apec1.O = 0.7
apec1.Ne = 1.2
apec1.Mg = 2.3
apec1.Si = 2.8
apec1.Fe = 1.6

# Normalization of apec model is VEM and some distance factor
# Using the same distance that Guenther & Schmitt (2009) used.
distance_factor = 1e-14 / (4 * np.pi * (122 * 3.085677581491367e+18)**2)

apec1.norm = distance_factor * 2.3e52
apec2.norm = distance_factor * 1.2e52
apec3.norm = distance_factor * 0.5e52


In [ ]:
src

Next, we call the `fake_all_orders` function with the wide order-sorting region and an exposure time of 100 ks. Then, we sit back and relax a little because, depending on the complexity of the model and the number of photons simulated, it might take a while.

<div style="background-color:orange;font-weight:bold">
If you use this notebook as a template, edit the path and exposure time in the cell below.</div>

In [ ]:
fake = fake_all_orders(src, arfrmfpath, 'osiptouch', exposure=1e5)

Below is a list of what we get. We can then either use [Sherpa datastack functions that operate on all these spectra simultaneously](https://cxc.cfa.harvard.edu/sherpa/ahelp/datastack.html) or use standard Sherpa on individual orders, e.g. `ui.plot_data('far_-9')`.

In [ ]:
fake.show_stack()

In [ ]:
# First, let's group all simulated spectra. Looks much better in plotting.
out = fake.group_counts(5)

In [ ]:
# I think grating spectra look better in wavelength space.
ui.set_analysis('wave')

Making a good plot of a high-resolution emission line spectrum is always a little bit of an issue because the dynamic range in the lines is so large. When you run this notebook yourself locally `%matplotlib notebook` will help, so that you can interactively zoom into the plots and look at the most important regions. However, when I render this for my website that won't work as well, so I'm selecting several (but not all!) orders that we simulated and show them below and then make another plot of just the O VII triplet that I'm interested in after that.

In [ ]:
orders_to_plot = ['0', 'near_+1', 'far_-9', 'far_-7', 'far_-6', 'far_-5', 'far_-3']

fig, axes = plt.subplots(nrows=len(orders_to_plot), figsize=(12, 12))
for ax, order in zip(axes, orders_to_plot): 
    plt.sca(ax)
    ui.plot_data(order, clearwindow=False)
    out = ax.set_title(f'Order: {order}')
    
fig.subplots_adjust(hspace=1)

All spectra are binned to a minimum of 5 counts per bin. Some orders (e.g. "0" or "near_+1") cover a pretty large range of wavelengths, while orders (e.g. "far_-9") only have a fairly narrow wavelength range. Of course, we could make the plots above such that they all share the same x-axis in $\lambda$ or in $m\lambda$, but I found it useful to just show what's in each order. Now, we plot the O VII triplet in particular, because that is one of the interesting features in this star. In XMM-Newton, the O VII triplet was seen in the coronal limit (large f/i ratio). However, HD 163296 is hot enough that its UV field should change the level populations through photo-excitation, and the fact that we don't see that points to an X-ray emission that is a few stellar radii *above* the stellar photosphere, possibly in the collimation zone of the jet. So, I want to make sure we get enough counts in the O VII triplet to reliably measure the line ratio.

In [ ]:
orders_to_plot = ['far_-6', 'far_-5']

fig, axes = plt.subplots(ncols=2, figsize=(12, 4))
for ax, order in zip(axes, orders_to_plot): 
    plt.sca(ax)
    ui.plot_data(order, clearwindow=False)
    out = ax.set_title(f'Order: {order}')
    ax.set_xlim(21.5, 22.2)
    
fig.subplots_adjust(hspace=1)

The O VII triplet is seen in two orders (-6 and -5). It's much stronger in order -6 than in -5 (look at the range of the y-axis), but nevertheless both orders contribute some signal, so, I probably want to fit them together. This is **not** the same as the interloping orders I discussed above. Order -6 contains true order -5, -6, and -7 photons, but the order -5 photons sorted into order -6 are so few that we don't see them and, because they are sorted into the wrong order, they would also be displayed at a different wavelength (and $5/6 * 21.6$ Å). Similarly, order -5 contains a few true order -6 photons, but we don't see them here. What we *do* see is simply that the orders overlap in wavelength space.

So, to determine if this 100 ks exposure is sufficient to reach my science goal, I want to fit the three lines in this triplet with three independent Gaussians, so that I can evaluate the error I get on the line ratio. (Since I'm really interested in the ratio and not in the flux itself, I might set up a model that is parameterized by the ratio instead of the flux, and possibly include a model for the continuum, too, but that is beyond the scope if this notebook.)

In [ ]:
rline = ui.normgauss1d('r')
iline = ui.normgauss1d('i')
fline = ui.normgauss1d('f')
o7model = rline + iline + fline

# Lines are unresolved, so fix FWHM at small number
for l in [rline, iline, fline]:
    l.fwhm = 0.001
    l.fwhm.frozen = True
    # Set some starting value close to final to 
    # speed up convergence and avoid
    # local minima
    l.ampl = 1e-5
    
rline.pos = 21.6
iline.pos = 21.8
fline.pos = 22.1

In [ ]:
out = fake.notice(21.5, 22.2)

In [ ]:
# set this new O VII model for all datasets in the stack
fake.set_source(o7model)
out = ui.fit('far_-6', 'far_-5')

In [ ]:
# Just show one of the fitted datasets here
ui.plot_fit('far_-6')

To get an idea how well we will be able to determine the ratio between the $f$ and $i$ line in the triplet, I run some Monte-Carlo simulations based on the fit above and the covariance determined. This may not be the best setup in detail (e.g. the simulations use a flat prior, while in reality the ratio is constrained to a certain range by atomic physics), but it is good enough to show that 100 ks will be more than enough to distinguish between an O VII emission with photo-excitation (on the stellar surface) and without it (above the stellar surface). Of course, the true spectrum could be more complicated, e.g. it could have two components at different locations, but that is beyond the scope of this notebook.

In [ ]:
# but to save time we fit only the two datsets that we know to have O VII data in them
# We don't have that many photons in each bin, so Cash is a better choice than default chi2gehrels
ui.set_stat('cash')
out = ui.fit('far_-6', 'far_-5')
# The MC sampler needs the covariance to select step sizes.
out = ui.covar('far_-6', 'far_-5')

In [ ]:
stats, accept, params = ui.get_draws(id='far_-6', otherids=('far_-5',), niter=1e4)

In [ ]:
fig, ax = plt.subplots()
f2i = params[5, :] / params[3, :]
f2i.sort()
ax.axhspan(0.05, 0.95, facecolor='0.8')
ax.plot(f2i, np.arange(len(f2i)) / len(f2i))
ax.set_xlabel('$f/i$ ratio')
out = ax.set_ylabel('cumulative fraction')

The figure shows that we expect the 90% confidence region (gray) for the observed $f/i$ ratio is about 3.0 to 5.0 and the small values of $f/i < 3$ can be excluded at a very high confidence level. That is good news, because photo-excitation would could these smaller values. We could improve this analysis (e.g. the upper limit has no meaning for this experiment because it is set purely by atomic physics, not by the emission region), but, again, that's beyond the scope of this notebook. We could also go back and re-run the simulation with a smaller exposure time, which will give us fewer counts and presumably lead to a larger uncertainty. What is the shortest exposure time that still allows us to distinguish the photo-excited from the "emission in the jet base" scenario?

## Summary

In this notebook, I show how to use Sherpa to simulate a single Arcus order with limited order sorting and how to loop over all orders to get a simulation of the full spectrum that we can expect from a real Arcus simulation. I run through one example to use the simulated spectrum to assess the feasibility of one particular science goal; this is meant as an example, not an exhaustive analysis and will have to modified depending on the science case in question.